In [28]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import date

In [29]:
c_to_f = lambda c :(9/5)*c + 32

In [30]:
#read in .xls to df
df=pd.read_excel("../trinidad_plot/TrinidadBeach_TEMP_1975-20230622.xls") #change excel filename
df = df[47:].reset_index().drop('index',axis=1)
df = df.drop(columns=['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])
df = df.rename(columns={'==================================================================================': 'year',
                        'Unnamed: 1': 'month',
                        'Unnamed: 2': 'day',
                        'Unnamed: 5': 'sst'})
df['date'] = pd.to_datetime(df[['year','month','day']])
df

,year,month,day,sst,date
0,1975,1,3,8,1975-01-03
1,1975,1,4,8.4,1975-01-04
2,1975,1,5,8.3,1975-01-05
3,1975,1,6,7.9,1975-01-06
4,1975,1,7,7.7,1975-01-07
...,...,...,...,...,...
17676,2023,5,27,NaN,2023-05-27
17677,2023,5,28,NaN,2023-05-28
17678,2023,5,29,NaN,2023-05-29
17679,2023,5,30,12.8,2023-05-30


In [31]:
df = pd.read_csv('../sst_hist/SIO_TEMP_20230105.csv')
df = df[['YEAR','MONTH','DAY','SURF_TEMP_C']]
df = df.rename(columns = {'YEAR': 'year', 'MONTH': 'month', 'DAY': 'day', 'SURF_TEMP_C': 'sst'})
df['date'] = pd.to_datetime(df[['year','month','day']])

In [32]:
long_term_mean = df.sst.mean()
df['sst_a'] = df.sst - long_term_mean
yearly_stats = pd.DataFrame(columns=['year','min_date','min_sst','mean_sst','max_date','max_sst'])
for year in df.year.unique():
    year_sst = df[df.year == year]
    yearly_min_sst = year_sst.sst.min()
    yearly_min_date = year_sst.iloc[np.where(year_sst.sst == yearly_min_sst)[0][0]].date
    yearly_mean_sst = year_sst.sst.mean()
    yearly_max_sst = year_sst.sst.max()
    yearly_max_date = year_sst.iloc[np.where(year_sst.sst == yearly_max_sst)[0][0]].date
    new_row = pd.DataFrame({'year': [year],
                        'min_sst': [yearly_min_sst],
                        'min_date': [yearly_min_date],
                        'mean_sst': [yearly_mean_sst],
                        'max_sst': [yearly_max_sst],
                        'max_date': [yearly_max_date]})
    yearly_stats = pd.concat([yearly_stats, new_row], ignore_index=True)
yearly_stats = yearly_stats.set_index('year')

In [33]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=yearly_stats.index, y=yearly_stats.mean_sst, mode='lines', name='Yearly Mean', line=dict(color='black'), showlegend=False))
# fig.add_trace(go.Scatter(x=yearly_stats.index, y=yearly_stats.min_sst, mode='lines', name='Yearly Min', line=dict(color='blue'), showlegend=False))
# fig.add_trace(go.Scatter(x=yearly_stats.index, y=yearly_stats.max_sst, mode='lines', name='Yearly Max', line=dict(color='red'), showlegend=False))

# Update traces to include markers and disable hovertemplate
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(title='Sea Surface Temperature at Scripps', yaxis_title='Sea Surface Temperature (°C)', hovermode="x unified")

# Save figure
fig.write_html('trinidad_beach_time_series.html')
fig